In [8]:
import warnings
warnings.filterwarnings('ignore')

import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

import os
import json
import shutil

# Data preparation

In [9]:
label_path = '../../label/'
image_path = '../../data/'

classes = {
    'arrow': 0,
    'crossing': 1,
    'fishbone': 2
}

In [10]:
list_annot_file = os.listdir(label_path)

In [11]:
WIDTH = 1640
HEIGHT = 560

In [12]:
def json_annot_to_df(file_path):
    file = open(file_path)
    data = json.load(file)
    
    location = data['_via_settings']['core']['default_filepath'].split('data\\')[1]
    
    file_names = []
    regions = []
    
    for _, value in data['_via_img_metadata'].items():
        if len(value['regions']) > 0:
            file_names.append(location + value['filename'])
            regions.append(value['regions'])
    
    return pd.DataFrame({'file_name': file_names, 'regions': regions})

In [13]:
def bound_boxes(points):
    min_x = np.min(points[:, 0])
    max_x = np.max(points[:, 0])
    min_y = np.min(points[:, 1])
    max_y = np.max(points[:, 1])

    return [min_x, max_x, min_y, max_y]

In [14]:
def get_data(file_path):
    values = json_annot_to_df(file_path).values
    
    file_names = []
    bboxes = []
    labels = []

    for value in values:
        file_name, regions = value
        for region in regions:
            try:
                annot = region['shape_attributes']
                label = region['region_attributes']['label']
                pts = np.array([[annot['all_points_x'][i], annot['all_points_y'][i]] for i in range(len(annot['all_points_x']))])
                bound = bound_boxes(pts)
                file_names.append(file_name)
                bboxes.append(bound)
                labels.append(label.split('\n')[0])
            except: pass
    
    return file_names, bboxes, labels

In [15]:
def pascal_voc_to_yolo_bbox(bbox, w, h):
    x_min, x_max, y_min, y_max = bbox
    x_center = ((x_max + x_min) / 2) / w
    y_center = ((y_max + y_min) / 2) / h
    
    width = (x_max - x_min) / w
    height = (y_max - y_min) / h
    
    return [x_center, y_center, width, height]

In [16]:
data = {
    'filename': [],
    'class_id': [],
    'bboxes': []
}

for annot_file in list_annot_file:
    file_names, bboxes, labels = get_data(label_path + annot_file)
    class_id = []
    for label in labels:
        class_id.append(classes.get(label))
        
    data['filename'].extend(file_names)
    data['bboxes'].extend(bboxes)
    data['class_id'].extend(class_id)
    
data = pd.DataFrame(data)

In [19]:
data['class_id'].value_counts()

1    198
0    127
2     29
Name: class_id, dtype: int64

In [19]:
train, test = train_test_split(data.filename.unique(), test_size=0.2 ,random_state=20)
train, val = train_test_split(train, test_size=0.15, random_state=20)

In [20]:
def copy_image(image_files, folder):
    for image in image_files:
        curr_path = os.path.join(image_path, image)
        new_image_path = os.path.join(folder, str(image).replace('\\', '__'))
        shutil.copy(curr_path, new_image_path)

In [21]:
try: 
    dataset_root = '../datasets'
    os.mkdir('../datasets/images')
    os.mkdir('../datasets/images/train')
    os.mkdir('../datasets/images/test')
    os.mkdir('../datasets/images/val')
    
    os.mkdir('../datasets/labels')
    os.mkdir('../datasets/labels/train')
    os.mkdir('../datasets/labels/test')
    os.mkdir('../datasets/labels/val')
    
except: pass

In [22]:
copy_image(train, folder='../datasets/images/train')
copy_image(test, folder='../datasets/images/test')
copy_image(val, folder='../datasets/images/val')

In [23]:
os.path.splitext(train[0])[0]

'driver_37_30frame\\05181919_0299.MP4\\04890'

In [24]:
def create_label_file(image_files, folder):
    for image in image_files:
        fileName = os.path.splitext(image)[0]
        label_file_name = str(fileName).replace('\\', '__')
        df = data[data['filename'] == image]
        with open(os.path.join(folder, f'{label_file_name}.txt'), 'w') as f:
            for i in range(0, len(df)):
                bbox = pascal_voc_to_yolo_bbox(df.iloc[i]['bboxes'], WIDTH, HEIGHT)
                bbox_text = " ".join(map(str, bbox))
                txt = str(df.iloc[i]['class_id']) + " " + bbox_text
                f.write(txt)
                if i != len(df) - 1:
                    f.write("\n")

In [25]:
create_label_file(train, '../datasets/labels/train')
create_label_file(test, '../datasets/labels/test')
create_label_file(val, '../datasets/labels/val')

In [26]:
object_data=f"""
train: images/train
val: images/val
test: images/test
nc: 3
names:
  0: arrow
  1: crossing
  2: fishbone
"""
with open('object_data.yaml', 'w') as f:
  f.write(object_data)

# Model Training

In [2]:
from ultralytics import YOLO

In [3]:
model = YOLO('yolov8n.pt')

In [21]:
model.train(data='object_data.yaml', epochs=50, batch=8)

Ultralytics YOLOv8.2.2  Python-3.10.11 torch-2.2.2+cpu CPU (11th Gen Intel Core(TM) i7-11800H 2.30GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=object_data.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, 

train: Scanning D:\code\py\lane_object_detection\src\datasets\labels\train.cache... 193 images, 0 backgrounds, 0 corrupt: 100%|██████████| 193/193 [00:00<?, ?it/s]
val: Scanning D:\code\py\lane_object_detection\src\datasets\labels\val.cache... 35 images, 0 backgrounds, 0 corrupt: 100%|██████████| 35/35 [00:00<?, ?it/s]

Plotting labels to runs\detect\train\labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50         0G      2.323      3.964      1.944          6        640: 100%|██████████| 25/25 [00:56<00:00,  2.24s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00,  1.01it/s]

...

                   all         35         38      0.856      0.809      0.868        0.7



50 epochs completed in 0.777 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train\weights\best.pt, 6.2MB

Validating runs\detect\train\weights\best.pt...
Ultralytics YOLOv8.2.2  Python-3.10.11 torch-2.2.2+cpu CPU (11th Gen Intel Core(TM) i7-11800H 2.30GHz)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  1.75it/s]


                   all         35         38      0.857      0.806      0.859      0.709
                 arrow         35          7      0.988      0.571      0.693      0.454
              crossing         35         30      0.927      0.847       0.89      0.777
              fishbone         35          1      0.657          1      0.995      0.895
Speed: 0.9ms preprocess, 31.1ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs\detect\train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000174B899EEC0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          

In [6]:
model.val(data='object_data.yaml')

Ultralytics YOLOv8.2.2  Python-3.10.11 torch-2.2.2+cpu CPU (11th Gen Intel Core(TM) i7-11800H 2.30GHz)


val: Scanning D:\code\py\lane_object_detection\src\datasets\labels\val.cache... 35 images, 0 backgrounds, 0 corrupt: 100%|██████████| 35/35 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  1.63it/s]


                   all         35         38      0.857      0.806      0.859      0.709
                 arrow         35          7      0.988      0.571      0.693      0.454
              crossing         35         30      0.927      0.847       0.89      0.777
              fishbone         35          1      0.657          1      0.995      0.895
Speed: 0.7ms preprocess, 32.5ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs\detect\val3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001BB7BB90A90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          

In [4]:
model = YOLO('runs/detect/train/weights/best.pt')

In [5]:
pred = model.predict('../datasets/images/train/driver_37_30frame__05181520_0219.MP4__03990.jpg', show=True)
cv2.waitKey()
cv2.destroyAllWindows()


image 1/1 d:\code\py\lane_object_detection\src\yolo\..\datasets\images\train\driver_37_30frame__05181520_0219.MP4__03990.jpg: 256x640 3 arrows, 70.9ms
Speed: 2.0ms preprocess, 70.9ms inference, 1235.0ms postprocess per image at shape (1, 3, 256, 640)


In [24]:
model.val(data='object_data.yaml')

Ultralytics YOLOv8.2.2  Python-3.10.11 torch-2.2.2+cpu CPU (11th Gen Intel Core(TM) i7-11800H 2.30GHz)


val: Scanning D:\code\py\lane_object_detection\src\datasets\labels\val.cache... 35 images, 0 backgrounds, 0 corrupt: 100%|██████████| 35/35 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  1.61it/s]


                   all         35         38      0.857      0.806      0.859      0.709
                 arrow         35          7      0.988      0.571      0.693      0.454
              crossing         35         30      0.927      0.847       0.89      0.777
              fishbone         35          1      0.657          1      0.995      0.895
Speed: 0.5ms preprocess, 31.3ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs\detect\val


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000174B5F91120>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          